# Séminaire IMT Grand-Est

# Introduction à l'apprentissage automatique - TP7 exercice 1


### Classification d'images à la fin du XXème siècle


<br>

Le but de ce TP est de découvrir une application de la vision par ordinateur, la classification d'images.

On utilisera un sous-ensemble de la [base de données Caltech101](http://www.vision.caltech.edu/Image_Datasets/Caltech101/), constitué d'images appartenant à 6 catégories, disponible [à cette URL](https://members.loria.fr/FSur/enseignement/apprauto/Caltech_small3.zip). La base de donnée est volontairement réduite pour rester gérable pendant un TP, mais vous pouvez bien entendu tenter de traiter la base entière sur votre temps libre.

<br>
Commencez par charger la base réduite dans un répertoire local, et visualisez les images qu'elle contient.

<br>

Le but est de prédire la catégorie d'une image de la base de test, connaissant les catégories auxquelles appartiennent les images de la base d'apprentissage. Il s'agit donc d'un problème de classification supervisée.

Les images ont des tailles différentes, de l'ordre de 300x200 pixels. Ce sont des images en couleur, un pixel est donc décrit par un triplet de composantes Rouge, Vert, Bleu. Les composantes prennent des valeurs entre 0 et 255 (codage sur 8 bits), de manière à ce que (0,0,0) est le noir, (255,255,255) le blanc, (0,255,0) le vert "pur", etc. Chaque image est donc décrite par approximativement 3x300x200=120000 valeurs variant entre 0 et 255. 

Le défi est de réduire la dimensionnalité du problème, en extrayant de chaque image une information pertinente représentée dans un espace de dimension "raisonnable", commun à toutes les images. Nous allons utiliser une méthode inspirée de celle présentée dans [cet article](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.459.9821&rep=rep1&type=pdf) très célèbre de 1999: chaque image sera représentée par un vecteur de ${\mathbb R}^{48}$ obtenu en concaténant les histogrammes sur 16 "baquets" (_bins_) de chaque composante R,V,B. Chaque composante étant codée sur 256 valeurs, un histogramme est donc obtenu en calculant la fréquence des valeurs entre 0 et 15, la fréquence des valeurs entre 16 et 31... d'où un histogramme sur 16 baquets.

<br>

On commence par importer des bibliothèques, et définir une fonction de visualisation de la base de test qui nous sera utile pour visualiser les performances des classifieurs.

In [ ]:
from sklearn import svm, model_selection, neural_network, neighbors, linear_model, metrics, cluster
from sklearn.metrics.pairwise import chi2_kernel
import numpy as np
#import time
import os
import matplotlib.pyplot as plt

def display_test(X_image_test,X_size_test,Y_test,Y_test_pred,txt):
    # paramètres: 
    # X_image_test: tableau des images de test représentées par un vecteur ligne
    # X_size_test: tableau des tailles des images de test
    # Y_test: classes d'appartenance réelles
    # Y_test_pred: classes prédites
    # txt: légende de la figure
    plt.figure(figsize=[16,10])    
    for n in range(60):
        plt.subplot(6,10,n+1,xticks=[],yticks=[])
        plt.imshow(X_image_test[n].reshape(X_size_test[n]),cmap='gray')
        if Y_test_pred[n]==Y_test[n]:
            plt.text(0.1,0.1,str(Y_test_pred[n])+' / '+str(Y_test[n]),fontsize=8,bbox=dict(facecolor='white', alpha=1))
        else:
            plt.text(0.1,0.1,str(Y_test_pred[n])+' / '+str(Y_test[n]),fontsize=8,bbox=dict(facecolor='red', alpha=1))
    plt.suptitle('prediction '+txt)
    plt.show();



<br>

La cellule suivante permet de lire les données et stocker l'information dans des tableaux. Constatez que chaque classe fournit 60 images d'apprentissage et 10 images de test.

In [ ]:
from PIL import Image  # on utilise la bibliothèque de manipulation d'image PIL pour redimensionner les images

# modifiez le chemin d'accès à vos données:
path="./Caltech_small3/" 

X_image_train=[] # tableau des images d'aprentissage, chaque ligne du tableau comprenant une image décrite ligne à ligne (voir la documentation de flatten dans numpy)
X_image_test=[]  # idem pour les images de test
X_imageresize_train=[]  # images redimensionnées (pas utilisées dans un premier temps)
X_imageresize_test=[]   # idem pour les images de test
X_size_train=[]  # tableau dont chaque ligne contient (nbre de lignes, nbre de colonnes) des images d'apprentissage
X_size_test=[]   # idem pour la base de test
X_train=[]       # tableau dont chaque ligne contient l'histogramme des niveaux de gris d'une image d'apprentissage
Y_train=[]       # tableau des classes d'appartenance des images
X_test=[]        # pour la base de test
Y_test=[]        # pour la base de test

y=0  # numéro de classe
for directory in os.listdir(path):
    count = 0 # indice de l'image traitée dans le répertoire courant
    print("%s  - classe: %d" % (directory,y))
    for file in os.listdir(path+directory):
        #img=plt.imread(path+directory+"/"+file)
        img=Image.open(path+directory+"/"+file)
        imgresize=np.array(img.resize((50,50))).flatten()/255
        img=np.array(img)
        imgflatten=np.array(img).flatten()
        #print(img.shape)
        if (count<60):
            X_size_train.append(img.shape)
            X_image_train.append(imgflatten)
            X_imageresize_train.append(imgresize)
            hR,bR=np.histogram(img[:,:,0],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
            hG,bG=np.histogram(img[:,:,1],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
            hB,bB=np.histogram(img[:,:,2],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
            X_train.append(np.concatenate((hR,hG,hB)))       
            Y_train.append(y)
        else: 
            if (count<70):
                X_size_test.append(img.shape)
                X_image_test.append(imgflatten)
                X_imageresize_test.append(imgresize)
                hR,bR=np.histogram(img[:,:,0],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
                hG,bG=np.histogram(img[:,:,1],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
                hB,bB=np.histogram(img[:,:,2],range=(0,255),density=True,bins=16)  # on calcule la fréquence des niveaux de gris dans l'image courante
                X_test.append(np.concatenate((hR,hG,hB)))   
                Y_test.append(y)
        count=count+1
    y=y+1

Selon vous, pourquoi utilise-t-on `density=True`, et quel est l'intérêt pour ces images de tailles différentes? 
(référez-vous à la documentatio de la fonction numpy [histogram](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html))

La cellule suivante permet de visualiser une image et les histogrammes (fréquences empiriques) de chaque canal. Expliquez le lien entre l'allure de l'histogramme et le contenu de l'image.

In [ ]:
print("%d observations dans la base d'apprentissage, chacune est décrite par un vecteur de dimension %d" % (len(X_train), len(X_train[0])) )
# exemple: visualisation d'une image
n_image=40  # indice d'une image de la base d'apprentissage, essayez d'autres valeurs
plt.figure()
plt.imshow(X_image_train[n_image].reshape(X_size_train[n_image]))  # reshape permet de transformer la représentation "en ligne" d'une image en une représentation 2D de taille nbre de lignes x nbre de colonnes
plt.title("image: %d" % n_image)
plt.figure()
plt.plot(X_train[n_image][0:16])
plt.title("histogramme canal R")
plt.figure()
plt.plot(X_train[n_image][16:32])
plt.title("histogramme canal G")
plt.figure()
plt.plot(X_train[n_image][32:48])
plt.title("histogramme canal B")

Testez différents algorithmes de classification sur les images brutes (bases  `X_image_train` et `X_image_test`): SVM, MLP, NN, LR.

<br>

* Pour les SVM, vous testerez les noyaux 'linear', 'rbf', et 'chi2_kernel'.

On peut utiliser une SVM avec un noyau du $\chi^2$ (on sait que cette métrique est bien adaptée à la comparaison d'histogrammes d'après le cours de statistique) à l'aide de:
```python 
SVM = svm.SVC(kernel=chi2_kernel)
``` 
avec l'import préalable:
```python
from sklearn.metrics.pairwise import chi2_kernel
```
Ce mode de définition d'un noyau rend la modification du paramètre $\gamma$ plus compliquée, on se contentera de la valeur par défaut.

Vous fixerez les valeurs des hyperparamètres ($C$ et le paramètre du noyau pour RBF) à l'aide de `GridSearchCV`.

<br>

* Pour les MLP, vous testerez un réseau à une couche cachée à 5 neurones et un autre avec deux couches cachées à 5 neurones, dont vous fixerez les hyperparamètres `alpha` et `max_iter` par `GridSearchCV`.

<br>

Pour chaque classifieur, vous afficherez le score de classification sur la base de test, la matrice de confusion, et la visualisation de la classification obtenue à l'aide de la fonction `display_test` fournie.

<br> 

Quel est le score de classification "minimal" auquel se comparer ?

In [ ]:

# gridsearch pour SVM

C_range=[1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
gamma_range=[1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]


SVM = svm.SVC(kernel='linear')
parameters={'C':C_range}
gridsearch=model_selection.GridSearchCV(SVM, parameters, cv=5)
gridsearch.fit(X_train,Y_train)
print("Meilleurs paramètres pour SVM_linear:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score']
plt.figure(figsize=[10,10])
plt.plot(C_range,scores)
plt.xlabel('C')
plt.ylabel("score")
plt.show();


SVM = svm.SVC(kernel=chi2_kernel)
parameters={'C':C_range}
gridsearch=model_selection.GridSearchCV(SVM, parameters, cv=5)
gridsearch.fit(X_train,Y_train)
print("Meilleurs paramètres pour SVM_chi2:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score']
plt.figure(figsize=[10,10])
plt.plot(C_range,scores)
plt.xlabel('C')
plt.ylabel("score")
plt.show();


SVM = svm.SVC(kernel='rbf')
parameters = { 'gamma':gamma_range , 'C':C_range }
gridsearch=model_selection.GridSearchCV(SVM, parameters, cv=5)
gridsearch.fit(X_train,Y_train)
print("Meilleurs paramètres pour SVM_rbf:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(gamma_range),len(C_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.colorbar()
plt.show();




In [ ]:

# gridsearch pour MLP

max_iter_range=[1000, 2500, 5000]
alpha_range=[1e-4, 1e-3, 1e-2, 1e-1]
parameters = {'solver': ['lbfgs'],  'alpha':alpha_range , 'max_iter':max_iter_range }

MLP = neural_network.MLPClassifier(hidden_layer_sizes=(20,))
gridsearch=model_selection.GridSearchCV(MLP, parameters, cv=5)
gridsearch.fit(X_train,Y_train)
print("Meilleurs paramètres pour MLP (20):")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(alpha_range),len(max_iter_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.ylabel('alpha')
plt.xlabel('max_iter')
plt.yticks(np.arange(len(alpha_range)), alpha_range, rotation=45)
plt.xticks(np.arange(len(max_iter_range)), max_iter_range)
plt.colorbar()
plt.show();

MLP = neural_network.MLPClassifier(hidden_layer_sizes=(5,5))
gridsearch=model_selection.GridSearchCV(MLP, parameters, cv=5)
gridsearch.fit(X_train,Y_train)
print("Meilleurs paramètres pour MLP (20,20):")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(alpha_range),len(max_iter_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.ylabel('alpha')
plt.xlabel('max_iter')
plt.yticks(np.arange(len(alpha_range)), alpha_range, rotation=45)
plt.xticks(np.arange(len(max_iter_range)), max_iter_range)
plt.colorbar()
plt.show();


In [ ]:
# classification


SVM = svm.SVC(kernel='linear',C=1000)
SVM.fit(X_train,Y_train)
print("score SVM linear : %.3f" % SVM.score(X_test, Y_test) )
Y_test_pred=SVM.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM linear")

SVM = svm.SVC(kernel='rbf',C=10,gamma=1000)
SVM.fit(X_train,Y_train)
print("score SVM rbf : %.3f" % SVM.score(X_test, Y_test) )
Y_test_pred=SVM.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM rbf")

SVM = svm.SVC(kernel=chi2_kernel,C=100)
SVM.fit(X_train,Y_train)
print("score SVM chi2 : %.3f" % SVM.score(X_test, Y_test) )
Y_test_pred=SVM.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM chi2")


MLP = neural_network.MLPClassifier(hidden_layer_sizes=(20,), solver='lbfgs', max_iter=1000, alpha=.01)               
MLP.fit(X_train,Y_train)
print("score MLP (5) %.3f" % MLP.score(X_test, Y_test) )
Y_test_pred=MLP.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"MLP (20)")

MLP = neural_network.MLPClassifier(hidden_layer_sizes=(20,20), solver='lbfgs', max_iter=500, alpha=.01)               
MLP.fit(X_train,Y_train)
print("score MLP (5,5) %.3f" % MLP.score(X_test, Y_test) )
Y_test_pred=MLP.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"MLP (20,20)")

LR = linear_model.LogisticRegression()
LR.fit(X_train,Y_train)
print("score LR %.3f" % LR.score(X_test, Y_test) )
Y_test_pred=LR.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"LR")


NN = neighbors.KNeighborsClassifier(n_neighbors=1)    
NN.fit(X_train,Y_train)
print("score NN %.3f" % NN.score(X_test, Y_test) )
Y_test_pred=NN.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"NN")

NN5 = neighbors.KNeighborsClassifier(n_neighbors=5)   
NN5.fit(X_train,Y_train)
print("score NN5 %.3f" % NN5.score(X_test, Y_test) )
Y_test_pred=NN5.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"NN5")


Quel algorithme d'apprentissage présente les meilleures performances ?

Quelle est la classe la plus "facile" à reconnaître, et pourquoi ? La plus difficile ?

Que dire des biais de la base d'apprentissage ?


<br>

## Remarques 

Comme on le voit dans l'[article](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.459.9821&rep=rep1&type=pdf) dont on s'inspire, la méthode proposée est en fait de considérer l'histogramme des triplets R,G,B en chaque pixel. Chaque image est donc décrite par un vecteur de ${\mathbb R}^{16^3} = {\mathbb R}^{4096}$.
Cf section III A de l'article. L'avantage par rapport à l'approche développée dans ce TP est de tenir compte des corrélations entre composantes R,G,B.

Remarquez également que différents noyaux sont testés pour les SVM de l'article de Chapelle et al.



<br>

Cette méthode correspond à l'état de l'art scientifique à la fin des années 1990.

En 2004-2005 sont apparus de nouveaux algorithmes basés sur la notion de "sac de mots visuels" dont une description accessible est [disponible ici](https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb) par exemple. Il s'agit d'une adaptation aux images des sacs de mots que l'on vous a présentés en [TP 1 exercice 2](https://members.loria.fr/FSur/enseignement/apprauto/TP1_ex2_sujet.html).

Depuis 2012, les méthodes basées sur les [réseaux de neurones convolutionnels](https://medium.com/@tifa2up/image-classification-using-deep-neural-networks-a-beginner-friendly-approach-using-tensorflow-94b0a090ccd4) obtiennent les meilleurs résultats. Elles sont l'objet de l'exercice 2.

<br>

Notez la remarque dans l'article de Chapelle et al. de 1999, Section IV A p. 1058 à propos du noyau du $\chi^2$: "_It is not known if the kernel satisfies Mercer’s condition_". Ceci a été démontré dans [cet article](https://people.eecs.berkeley.edu/~malik/papers/FBCM-nystrom.pdf) de 2004 (annexe B).

<br>

### Alternative ?


Au lieu de décrire les images par des histogrammes, on pourrait envisager de normaliser la dimension des images, et de les décrire directement par les valeurs des trois composantes. C'est ce qui est fait lorsqu'on définit les variables `X_imageresize_train` et `X_imageresize_test`, pour lesquelles les images ont été redimensionnées à $50\times 50$ pixels. Bien entendu, la dimensionnalité du problème de classification est bien plus grande qu'avec les histogrammes. Cela justifie d'utiliser un plus grand nombre de neurones dans les couches cachées pour le MLP.

<br>
Les cellules suivantes montrent les résultats de classification obtenus par nos méthodes préférées. On évitera une _Grid search_ étant donné le temps d'exécution. 

In [ ]:
# résultats de la classification sur les images redimensionnées


SVM = svm.SVC(kernel='linear',C=.001)
SVM.fit(X_imageresize_train,Y_train)
print("score SVM linear : %.3f" % SVM.score(X_imageresize_test, Y_test) )
Y_test_pred=SVM.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM linear")

SVM = svm.SVC(kernel='rbf',C=10,gamma=.001)
SVM.fit(X_imageresize_train,Y_train)
print("score SVM rbf : %.3f" % SVM.score(X_imageresize_test, Y_test) )
Y_test_pred=SVM.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM rbf")

from sklearn.metrics.pairwise import chi2_kernel
SVM = svm.SVC(kernel=chi2_kernel,C=.001)
SVM.fit(X_imageresize_train,Y_train)
print("score SVM chi2 : %.3f" % SVM.score(X_imageresize_test, Y_test) )
Y_test_pred=SVM.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"SVM chi2")


MLP = neural_network.MLPClassifier(hidden_layer_sizes=(100,), solver='lbfgs', max_iter=1500, alpha=.001)               
MLP.fit(X_imageresize_train,Y_train)
print("score MLP (5) %.3f" % MLP.score(X_imageresize_test, Y_test) )
Y_test_pred=MLP.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"MLP (100)")

MLP = neural_network.MLPClassifier(hidden_layer_sizes=(100,100), solver='lbfgs', max_iter=1500, alpha=.001)               
MLP.fit(X_imageresize_train,Y_train)
print("score MLP (5,5) %.3f" % MLP.score(X_imageresize_test, Y_test) )
Y_test_pred=MLP.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"MLP (100,100)")


LR = linear_model.LogisticRegression()
LR.fit(X_imageresize_train,Y_train)
print("score LR %.3f" % LR.score(X_imageresize_test, Y_test) )
Y_test_pred=LR.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"LR")


NN = neighbors.KNeighborsClassifier(n_neighbors=1)    
NN.fit(X_imageresize_train,Y_train)
print("score NN %.3f" % NN.score(X_imageresize_test, Y_test) )
Y_test_pred=NN.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"NN")


NN5 = neighbors.KNeighborsClassifier(n_neighbors=5)   
NN5.fit(X_imageresize_train,Y_train)
print("score NN5 %.3f" % NN5.score(X_imageresize_test, Y_test) )
Y_test_pred=NN5.predict(X_imageresize_test)
print(metrics.confusion_matrix(Y_test,Y_test_pred))
display_test(X_image_test,X_size_test,Y_test,Y_test_pred,"NN5")
